In [1]:
import matplotlib.pyplot as plt
import seaborn as sns

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.tree import export_graphviz
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix

import descartes
import geopandas as gpd
from shapely.geometry import Point, Polygon
import folium

from scipy import stats

### Acquire the Merged.csv of the health and Yelp data

In [2]:
df=pd.read_csv('merged.csv')
pd.set_option('display.max_columns', None)

In [3]:
df=pd.DataFrame(df)
df.head()
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2409 entries, 0 to 2408
Data columns (total 29 columns):
Unnamed: 0              2409 non-null int64
inspection_date         2409 non-null object
inspector               2409 non-null object
number_of_violations    2409 non-null int64
violations              2409 non-null object
zip_code                2409 non-null int64
house_numbers           2409 non-null int64
categories              2409 non-null object
transactions            2409 non-null object
distance                2409 non-null float64
review_count            2409 non-null float64
longitude               2409 non-null float64
latitude                2409 non-null float64
name                    2409 non-null object
sector                  2409 non-null int64
district                2409 non-null object
total_score             2409 non-null int64
rating                  2409 non-null float64
price                   2409 non-null object
establishment_x         2409 non-null o

In [4]:
# Dropping redundant and unneccesary columns

df= df.drop(df[['transactions', 'price', 'establishment_x', 'establishment_y', 'address_x', 'address_y', 'trimmed_name_x', 'trimmed_name_y', 'clean_address_x', 'clean_address_y', 'address_distance', 'name_distance']], axis=1)
df.describe()

,Unnamed: 0,number_of_violations,zip_code,house_numbers,distance,review_count,longitude,latitude,sector,total_score,rating
count,2409.000000,2409.000000,2409.000000,2409.000000,2409.000000,2409.000000,2409.000000,2409.000000,2409.000000,2409.000000,2409.000000
mean,1204.000000,4.989622,78220.360731,4494.122042,7806.578807,161.892902,-98.514704,29.486358,16.587796,94.044832,3.574720
std,695.562722,4.708035,14.884181,4491.037788,3305.930381,264.744464,0.059607,0.055306,9.608700,5.482347,0.769284
min,0.000000,0.000000,78201.000000,1.000000,811.199667,1.000000,-98.708442,29.354318,1.000000,56.000000,1.000000
25%,602.000000,2.000000,78209.000000,849.000000,5840.653743,22.000000,-98.564331,29.430632,8.000000,91.000000,3.000000
50%,1204.000000,4.000000,78216.000000,2727.000000,7940.300340,69.000000,-98.499740,29.486788,17.000000,95.000000,3.500000
75%,1806.000000,7.000000,78230.000000,7400.000000,9571.434425,186.000000,-98.480582,29.526893,22.000000,98.000000,4.000000
max,2408.000000,50.000000,78259.000000,22211.000000,20222.717722,2661.000000,-98.397780,29.661520,35.000000,100.000000,5.000000


In [5]:
#Dropped all establishments with less than 22 reviews 
# df=  df[df['review_count'] >= 22] 
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2409 entries, 0 to 2408
Data columns (total 17 columns):
Unnamed: 0              2409 non-null int64
inspection_date         2409 non-null object
inspector               2409 non-null object
number_of_violations    2409 non-null int64
violations              2409 non-null object
zip_code                2409 non-null int64
house_numbers           2409 non-null int64
categories              2409 non-null object
distance                2409 non-null float64
review_count            2409 non-null float64
longitude               2409 non-null float64
latitude                2409 non-null float64
name                    2409 non-null object
sector                  2409 non-null int64
district                2409 non-null object
total_score             2409 non-null int64
rating                  2409 non-null float64
dtypes: float64(5), int64(6), object(6)
memory usage: 320.0+ KB


#### Grouping Health Scores into bins

In [6]:
#grouped health scores into 4 groups
health_scores= {'health_scores' : []}
for x in df['total_score']:
#     print(x)
    if x > 99:
        health_scores['health_scores'].append('1')
    elif x>=96:
        health_scores['health_scores'].append('2')
    elif x >=92:
        health_scores['health_scores'].append('3')
    else :
        health_scores['health_scores'].append('4')
#df
df['health_scores']= health_scores['health_scores']  

## Checking the distribution of health scores

In [7]:
df['health_scores'].value_counts()

2    697
4    657
3    608
1    447
Name: health_scores, dtype: int64

In [8]:
#encode for the inspectors
df['inspector'].value_counts()

Lauren Schumacher     194
Jacob Johanson        159
RYAN DAVIS            153
Jeffrey Carrizales    148
Sumner Bumgardner     127
Diana Garcia          117
Cicily Martinez       116
Bamba Njie            109
Bernard Capdeboscq    105
Valerie Suarez        104
Benito Zuniga          97
Deborah Liu            96
Nathan Balfour         90
Raul Jimenez           87
Daisy Zamora           82
Debra Hernandez        59
MICHELLE MORENO        55
Mario Cavazos          54
Kassandra Aguilar      53
Amanda Cantu           52
John Payelle           48
William Hudson         43
Erica Arreola          42
William Emminger       37
Kathleen Prenzler      34
Adan Zamora            24
Monica Caballero       22
Sean Gilbert           20
Kacie Patrick          20
Fermin Garza           18
Joe Mokry              15
Juanicia Page          10
Victoria Paredes        9
Daniel Guzman           4
Maurico Ripley          3
Erica Llanas            2
Rebecca Vera            1
Name: inspector, dtype: int64

In [9]:
#Dropping inspectors that have not copmpleted AT LEAST 2 inspections
vdf = pd.DataFrame(df['inspector'].value_counts())
vdf['check'] = vdf['inspector'] > 1
over_20 = list(vdf.loc[vdf['check'], :].reset_index()['index'])

In [11]:
df['check'] = df['inspector'].isin(over_20)
new_df = df.loc[df['check'], :]
new_df = new_df.drop(columns = ['check', 'Unnamed: 0'])
new_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2408 entries, 0 to 2408
Data columns (total 17 columns):
inspection_date         2408 non-null object
inspector               2408 non-null object
number_of_violations    2408 non-null int64
violations              2408 non-null object
zip_code                2408 non-null int64
house_numbers           2408 non-null int64
categories              2408 non-null object
distance                2408 non-null float64
review_count            2408 non-null float64
longitude               2408 non-null float64
latitude                2408 non-null float64
name                    2408 non-null object
sector                  2408 non-null int64
district                2408 non-null object
total_score             2408 non-null int64
rating                  2408 non-null float64
health_scores           2408 non-null object
dtypes: float64(5), int64(5), object(7)
memory usage: 338.6+ KB


## Encoding sector data to add geographical reference to inspector data

## Total Entries is 2408

In [ ]:
#Encoding the inspector column for modeling.
label_enc= LabelEncoder()
new_df["sector_encode"] = label_enc.fit_transform(new_df.sector)


In [ ]:
#OneHot Encoding
one_enc = OneHotEncoder()
Y = one_enc.fit_transform(new_df.sector_encode.values.reshape(-1,1)).toarray()
# df_array = np.array(new_long_df.encoded).reshape(len(new_long_df.encoded),1)


In [ ]:
#Joining the encoded data to the main Df
# elist = [X,Y,Z]
dfOneHot = pd.DataFrame(Y, columns = ["sector_"+str(int(i)) for i in range(Y.shape[1])])

In [ ]:
new_df = new_df.reset_index()
new_df = new_df.reset_index()
new_df['index']= new_df['level_0']
new_df = new_df.drop(columns = ['level_0'])
dfOneHot = dfOneHot.reset_index()
enc_df=new_df.join(dfOneHot, on = 'index', lsuffix='_left', rsuffix='_right')

### Decision Tree Model

In [ ]:
#### Decision tree including number of review, total violations, yelp score, and inspector
#training / test split
X= enc_df.drop(columns=[ 'index_left', 'index_right', 'inspection_date','longitude', 'latitude',  'inspector', 'number_of_violations','violations', 'zip_code', 'house_numbers', 'categories', 'distance', 'name', 'sector', 'district', 'health_scores', 'total_score', 'sector_encode'])
y= enc_df['health_scores']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = .3, random_state = 42, stratify=enc_df.health_scores)
X_train.head()


In [ ]:
#Train Model

#Create the Decision Tree Object


clf = DecisionTreeClassifier(criterion='entropy', max_depth=4, random_state= 42, min_samples_split= 2)

In [ ]:
#Fit the model to the training data

clf.fit(X_train, y_train)

In [ ]:
#Estimate health group

y_pred = clf.predict(X_train)
y_pred

In [ ]:
#Estimate the probability of a health group
y_pred_proba = clf.predict_proba(X_train)
y_pred_proba

In [ ]:
#Create a confusion matrix
confusion_matrix(y_train, y_pred)

In [ ]:
#Accuracy: number of correct predictions over the number of total instances that have been evaluated.
print('Accuracy of Decision Tree classifier on training set: {:.2f}'
     .format(clf.score(X_train, y_train)))

In [ ]:
#Create a classification report
print(classification_report(y_train, y_pred))

In [ ]:
#Test Model

#Compute the accuracy of the model when run on the test data

print('Accuracy of Decision Tree classifier on test set: {:.2f}'
     .format(clf.score(X_test, y_test)))

In [ ]:
#Testing the accuracy with chi square

#Health scores and inspector

contingency_table = pd.crosstab(enc_df.inspector, enc_df.health_scores)
chi2_stat, p_val, dof, ex = stats.chi2_contingency(contingency_table)
print("===Chi2 Stat===")
print(chi2_stat)
print("\n")
print("===Degrees of Freedom===")
print(dof)
print("\n")
print("===P-Value===")
print(p_val)
print("\n")


### Decision Tree Model- Binning based on probability

In [ ]:
#created a table to show the probability of inspectors assigning a certain health score

contingency_table = pd.crosstab(df.inspector, df.health_scores, margins=True)
contingency_table = contingency_table[contingency_table.All > 20]
contingency_table['less_92'] = round(contingency_table['4']/contingency_table['All'],2)
contingency_table['92_to_95'] = round(contingency_table['3']/contingency_table['All'],2)
contingency_table['96_to_99'] = round(contingency_table['2']/contingency_table['All'],2)
contingency_table['100'] = round(contingency_table['1']/contingency_table['All'],2)
contingency_table.reset_index(inplace = True)
contingency_table

In [ ]:
df = enc_df.merge(contingency_table)

In [ ]:
df.columns

In [ ]:
df = df[['name','review_count','rating','less_92','92_to_95','96_to_99','100', 'health_scores','sector_0', 'sector_1', 'sector_2', 'sector_3',
       'sector_4', 'sector_5', 'sector_6', 'sector_7', 'sector_8', 'sector_9',
       'sector_10', 'sector_11', 'sector_12', 'sector_13', 'sector_14',
       'sector_15', 'sector_16', 'sector_17', 'sector_18', 'sector_19',
       'sector_20', 'sector_21', 'sector_22', 'sector_23', 'sector_24',
       'sector_25', 'sector_26', 'sector_27', 'sector_28', 'sector_29',
       'sector_30', 'sector_31', 'sector_32', 'sector_33']]
df.head()

## Splitting the Data

In [ ]:
#training / test split
X= df.drop(columns=['name','health_scores'])
y= df['health_scores']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = .3, random_state = 42, stratify=df.health_scores)
X_train.head()

In [ ]:
#Train Model

#Create the Decision Tree Object

clf = DecisionTreeClassifier(criterion='entropy', max_depth=5, random_state= 42, min_samples_split= 2)

In [ ]:
#Fit the model to the training data

clf.fit(X_train, y_train)

In [ ]:
#Estimate health group

y_pred = clf.predict(X_train)
y_pred

In [ ]:
#Estimate the probability of a health group
y_pred_proba = clf.predict_proba(X_train)
y_pred_proba

In [ ]:
confusion_matrix(y_train, y_pred)

In [ ]:
#Accuracy: number of correct predictions over the number of total instances that have been evaluated.
print('Accuracy of Decision Tree classifier on training set: {:.2f}'
     .format(clf.score(X_train, y_train)))

In [ ]:
#Test Model

#Compute the accuracy of the model when run on the test data

print('Accuracy of Decision Tree classifier on test set: {:.2f}'
     .format(clf.score(X_test, y_test)))

In [ ]:
print(classification_report(y_train, y_pred))

### Creating a Random Forrest Model and setting the estimator to one million, because why not

In [ ]:
#Train Model

#Create the Random Forest Object
#
rf = RandomForestClassifier(max_depth=4, criterion='entropy', n_estimators=100000, random_state=42)
#Fit the model to the training data
rf.fit(X_train, y_train)

In [ ]:
#Print the Feature importance
print(rf.feature_importances_)

In [ ]:
#Estimate the probability of the health group, using the training data
rf.score(X_train, y_train)

In [ ]:
print(classification_report(y_train, y_pred))

In [ ]:
#Compute the accuracy of the model when run on the test data

print('Accuracy of random forest classifier on test set: {:.2f}'
     .format(rf.score(X_test, y_test)))

# Creating a Confusion Matrix To Better Understand the Decision Tree Model

In [ ]:
confusion_matrix(y_train,y_pred)

In [ ]:
#Making the confusion matrix "pretty"
labels=(" 100 ", " 96-99 ", " 92-95 ", " <92 ")
pretty_cm=pd.DataFrame(confusion_matrix(y_train,y_pred),index=labels,columns=labels)
pretty_cm.T

## Takeaway: There is correlation between inspectors and the scores they give.
### Further exploration is warranted. The distribution of missed predictions highlights one of the drawbacks to the model, which was the inconsistency of restaurant categorization of each individual restaurant in Yelp Fusion. Having the categories encoded and used as features would produce an improved model. 
